In [1]:
import pandas as pd
from tqdm import tqdm 
def process_dataframe(df, output_file):
    # Open the output file in write mode and create a file object to write to.
    with open(output_file, 'w', encoding='utf-8') as f:
        # Write the YAML header to the output file.
        f.write('name: typofixer-da\n')
        f.write('parent: default\n')
        f.write('matches:\n')
        
        # Iterate over each row in the input DataFrame.
        for i, row in df.iterrows():
            # Extract values from the row and remove leading/trailing whitespace.
            trigger = row[0].strip()
            replace = row[1].strip()
            word = str(row[2]).lower()
            propagate_case = str(row[3]).lower()
            test_val = row[4]
            
            # Write the appropriate YAML output to the output file based on the value of test_val.
            if test_val:
                f.write(f"- trigger: \"{trigger}\"\n")
                f.write(f"  replace: \"{replace}\"\n")
                f.write(f"  word: {word}\n")
                f.write(f"  propagate_case: {propagate_case}\n")
            else:
                f.write(f"- triggers: {trigger}\n")
                f.write(f"  replace: \"{replace}\"\n")
                f.write(f"  word: {word}\n")
                f.write(f"  propagate_case: {propagate_case}\n")


In [2]:
path = r'C:\Users\jbay\AppData\Roaming\espanso\match\packages\typofixer-da\package.yml'

data = open(path, encoding='utf-8').read().split('\n')

df = pd.DataFrame(columns=['trigger','replace', 'propagate_case', 'word', 'trigger=1'])
df

for i in tqdm(range(len(data))):
    if '- triggers' in data[i]:
        trigger = data[i].replace('- triggers: ', '')
        replace = data[i + 1].replace('  replace: ', '').replace('"','')
        word = data[i + 2].replace('  word: ','')
        prop = data[i + 3].replace('  propagate_case: ','')
        more = 0

        line = [trigger, replace, prop, word, more]
        df.loc[len(df)] = line 
    elif '- trigger' in data[i]:
        trigger = data[i].replace('- trigger: ', '').replace('"','')
        replace = data[i + 1].replace('  replace: ', '').replace('"','')
        word = data[i + 2].replace('  word: ','')
        prop = data[i + 3].replace('  propagate_case: ','')
        more = 1
        
        
        line = [trigger, replace, prop, word, more]
        df.loc[len(df)] = line 
        
df

100%|██████████| 320/320 [00:00<00:00, 2964.28it/s]


,trigger,replace,propagate_case,word,trigger=1
0,"[""abbonnement"",""abbonement"",""abonement""]",abonnement,true,true,0
1,"[""accellerere"",""acelerere""]",accelerere,true,true,0
2,addresse,adresse,true,true,1
3,afhængelig,afhængig,true,true,1
4,afspassere,afspadsere,true,true,1
...,...,...,...,...,...
74,"[""terasse"",""terresse"",""teresse""]",terrasse,true,true,0
75,udemærket,udmærket,true,true,1
76,ynglingsfilm,yndlingsfilm,true,true,1
77,"[""yugurt"",""yughurt"",""yougurt"",""youghurt"",""yogu...",yoghurt,true,true,0


In [29]:
new_df = pd.read_csv('top_1000.csv', index_col=0, delimiter=";")
new_df

,correct_spelling,misspellings
1,som,"['smo' , 'osm' , '..."
3,hans,"['hasn' , 'hnas' , 'h..."
4,at,"['ta'] , , ..."
5,han,"['hna' , 'ahn' , '..."
6,var,"['vra' , 'avr' , '..."
...,...,...
996,næse,"['næes' , 'nsæe' , 'nse..."
997,flertal,"['lertal' , 'fertal' , 'flrtal..."
998,vrede,"['rede' , 'vede' , 'vrde..."
999,fordring,"['ordring' , 'frdring' , 'fodring..."


In [30]:
new_words =  {"åp": "på",
              "dte": "det",
              "poc": "Proof of Concept",
              "teknsiske": "tekniske",
              "teknsiske": "tekniske",
}

# new_df = pd.read_csv(r'C:\Users\jbay\AppData\Roaming\espanso\match\scripts\to_add.csv', delimiter=";")
# for each in new_df[new_df.language == 'danish'].index:
for each in new_df.correct_spelling:
    trigger = each
    if trigger in new_words.keys() or trigger in df['replace'].values:
        pass
    else:
        new_words[trigger] = new_df.loc[new_df.correct_spelling == each, 'misspellings'].values[0]

In [31]:
each

'kontinent'

In [32]:
new_words

{'åp': 'på',
 'dte': 'det',
 'poc': 'Proof of Concept',
 'teknsiske': 'tekniske',
 'som': "['smo'                     , 'osm'         , 'oms'         , 'mso'         , 'mos']        ,               ,               ,               ,               ,               ,               ,               ,                ,          ,        ,       ,       ,       ,       ,        ,       ,        ,",
 'hans': "['hasn'                   , 'hnas'        , 'hnsa'        , 'hsan'        , 'hsna'        , 'ahns'        , 'ahsn'        , 'anhs'        , 'ansh'        , 'ashn'        , 'asnh'        , 'nhas'        , 'nhsa'         , 'nahs'   , 'nash' , 'nsha', 'nsah', 'shan', 'shna', 'sahn' , 'sanh', 'snha' , 'snah']",
 'at': "['ta']                      ,               ,               ,               ,               ,               ,               ,               ,               ,               ,               ,               ,                ,          ,        ,       ,       ,       ,       ,      

In [16]:
new_words

{'åp': 'på',
 'dte': 'det',
 'poc': 'Proof of Concept',
 'teknsiske': 'tekniske',
 'som': "['smo', 'osm', 'oms', 'mso', 'mos']"}

In [ ]:
        
for each in new_words.keys():
    for each in new_words.keys():
        if each in df.trigger.unique():
            pass
        else:
            df.loc[len(df)] = [each, new_words[each], 'true', 'true', 'true']
            
df.sort_values('replace', inplace=True, key=lambda x: x.str.lower() if x.dtype == 'object' else x)
df = df.reset_index()
df.drop(['index'], axis=1, inplace=True)
#df.to_csv(path, encoding='utf-8')
process_dataframe(df, "package.yml")

In [7]:
process_dataframe(df, "package.yml")

In [69]:
# import pyperclip

# new_line = pyperclip.paste() + "; ;unknown\n"

# first_lines = open('to_add.txt', "r").read()
# textfile    = open('to_add.txt', 'w')
# textfile.write(first_lines)
# textfile.write(new_line)
# textfile.close()